In [ ]:
import pandas as pd
import mplfinance as mpf
import matplotlib.patches as patches

# === Load OHLC data ===
ohlc = pd.read_csv("applyiedMacd-1.csv", parse_dates=["DateTime"])
ohlc.set_index("DateTime", inplace=True)

# === Load trades CSV ===
trades = pd.read_csv("bt.result.csv", parse_dates=["Entry Time", "Exit Time"])

# === Limit to 250 rows for plotting ===
ohlc_limited = ohlc.iloc[:250]

In [ ]:
import matplotlib.patches as patches

def draw_trade_boxes(ax, ohlc, trades):
    for _, trade in trades.iterrows():
        entry_idx = int(trade['Candle Index Entry'])
        exit_idx  = int(trade['Candle Index Exit'])

        # Skip if trade is outside the plotted window
        if entry_idx >= len(ohlc) or exit_idx >= len(ohlc):
            continue

        # X-axis: index positions (candles)
        x0 = entry_idx
        width = max(1, exit_idx - entry_idx)  # at least width=1

        # Y-axis: span from SL to TP
        y0 = min(trade['SL'], trade['TP'])
        height = abs(trade['TP'] - trade['SL'])

        # Decide box color:
        if trade['Exit Price'] == trade['SL']:
            facecolor = "red"   # SL hit
        elif trade['Exit Price'] == trade['TP']:
            facecolor = "green" # TP hit
        else:
            facecolor = "gray"  # neither (manual exit, maybe)

        # Draw rectangle (hollow box)
        rect = patches.Rectangle(
            (x0, y0),
            width,
            height,
            linewidth=1.5,
            edgecolor=facecolor,
            facecolor="none",
            linestyle="--"
        )
        ax.add_patch(rect)


In [ ]:
# === Plot with mplfinance and pass axes to custom function ===
fig, axlist = mpf.plot(
    ohlc_limited,
    type='candle',
    style='yahoo',
    figsize=(16,8),
    title='Backtest Trades with Boxes',
    returnfig=True
)

# Main candlestick axis is the first one
ax = axlist[0]

# Draw trade boxes
draw_trade_boxes(ax, ohlc_limited, trades)

mpf.show()